In [1]:
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
df_ipcr = pd.read_csv("./data/ipcr.tsv", delimiter="\t")

In [3]:
df_ipcr.head(5)

,uuid,patent_id,classification_level,section,ipc_class,subclass,main_group,subgroup,symbol_position,classification_value,classification_status,classification_data_source,action_date,ipc_version_indicator,sequence
0,00005z3qh82fwpo5r1oupwpr3,6864832,NaN,G,01,S,013,/42,NaN,NaN,NaN,NaN,NaN,NaN,0
1,0000662nssr53hdo3lp92sz26,9954111,A,H,01,L,27,1156,L,I,B,H,2018-04-24,2017-01-01,8
2,00008u9j3g8oivqtuc1dqayb1,10048897,A,G,06,F,12,891,L,I,B,H,2018-08-14,2016-01-01,7
3,00008v5gnw215cdjozwehxqky,10694566,A,H,4,W,4,0,F,I,B,H,2020-06-23,2018-01-01,0
4,0000hj3ytmy8g9l2qa5x1hta5,D409748,NaN,D,24,04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
df_ipcr["section"].isnull().sum()

0

In [5]:
df_ipcr["ipc_class"].isnull().sum()

0

In [6]:
df_ipcr["subclass"].isnull().sum()

15

In [7]:
df_ipcr = df_ipcr[ df_ipcr["subclass"].isnull() == False]

In [8]:
df_ipcr["ipc_class"] = df_ipcr["ipc_class"].map(str)
df_ipcr = df_ipcr[df_ipcr["ipc_class"].map(str.isdigit)]
df_ipcr["ipc_class"] = df_ipcr["ipc_class"].astype(int)

In [9]:
two_digit_f = lambda x : '{0:02d}'.format(x)
two_digit_f(3)

'03'

In [10]:
df_ipcr["ipc_class"] = df_ipcr["ipc_class"].map(two_digit_f)

In [11]:
df_ipcr["ipc_class"][:3]

0    01
1    01
2    06
Name: ipc_class, dtype: object

In [12]:
df_ipcr["subclass"] = df_ipcr["subclass"].map(str.upper)
df_ipcr = df_ipcr[df_ipcr["subclass"].isin(list("ABCEDFGHIJKLMNOPQRSTUVWXYZ"))]

In [13]:
df_ipcr["4digit"] = df_ipcr["section"] + df_ipcr["ipc_class"] + df_ipcr["subclass"]

In [14]:
df_data = df_ipcr[["patent_id", "4digit"]]
df_data.describe()

,patent_id,4digit
count,16723552,16723552
unique,6784478,4265
top,6846516,G06F
freq,211,1245259


In [15]:
df_data[:5]

,patent_id,4digit
0,6864832,G01S
1,9954111,H01L
2,10048897,G06F
3,10694566,H04W
5,7645556,G03F


In [16]:
df_data = df_data.drop_duplicates(['patent_id', '4digit'])

In [17]:
df_data.describe()

,patent_id,4digit
count,9821166,9821166
unique,6784478,4265
top,10315897,G06F
freq,24,717749


In [18]:
df_data["value"] = True

In [19]:
df_small_data = df_data.loc[:50000, :]

In [20]:
df_matrix = pd.crosstab(df_small_data.patent_id, df_small_data["4digit"], df_small_data.value, aggfunc="first")

In [21]:
df_matrix.fillna(0)

4digit,A01B,A01C,A01D,A01F,A01G,A01H,A01J,A01K,A01M,A01N,...,H61K,I22C,K06F,K11C,L08J,M03G,N61W,Q11C,R26B,R64C
patent_id,,,,,,,,,,,,,,,,,,,,,
10000050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000304,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RE48015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RE48050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T101101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_matrix

4digit,A01B,A01C,A01D,A01F,A01G,A01H,A01J,A01K,A01M,A01N,...,H61K,I22C,K06F,K11C,L08J,M03G,N61W,Q11C,R26B,R64C
patent_id,,,,,,,,,,,,,,,,,,,,,
10000050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RE48015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RE48050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T101101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_matrix.loc["3930477", :].argmax

<bound method IndexOpsMixin.argmax of 4digit
A01B    NaN
A01C    NaN
A01D    NaN
A01F    NaN
A01G    NaN
       ... 
M03G    NaN
N61W    NaN
Q11C    NaN
R26B    NaN
R64C    NaN
Name: 3930477, Length: 674, dtype: object>